# Final Exam Instructions
---

Write a Python program that takes as input a file containing DNA sequences in multi-FASTA format, and computes the answers to the following questions. You can choose to write one program with multiple functions to answer these questions, or you can write several programs to address them. We will provide a multi-FASTA file for you, and you will run your program to answer the exam questions. 

While developing your program(s), please use the following example file to test your work: dna.example.fasta

You'll be given a different input file to launch the exam itself.

Here are the questions your program needs to answer. The quiz itself contains the specific multiple-choice questions you need to answer for the file you will be provided.

---
1. How many records are in the file? A record in a FASTA file is defined as a single-line header, followed by lines of sequence data. The header line is distinguished from the sequence data by a greater-than (">") symbol in the first column. The word following the ">" symbol is the identifier of the sequence, and the rest of the line is an optional description of the entry. There should be no space between the ">" and the first letter of the identifier. 
---


In [37]:
def readFasta(filename):
    '''
    Read multiline fasta sequence from file.
    Input : fasta filename or file path.
    Output : dict with sequence name and fasta sequence.
    '''
    try:
        fastaFile = open(filename, "r")
    except IOError:
        print("File ",filename," does not exist!!")
        return {}
    
    seqs={}
    for line in fastaFile:
        line=line.rstrip()
        if line[0]=='>':
            words = line.split()
            name=words[0][1:]
            seqs[name]=""
        else : # sequence, not header
            seqs[name] = seqs[name] + line
    fastaFile.close()
    return seqs


In [48]:
dna = fasta['gi|142022655|gb|EQ086233.1|45']
dna

'CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGAAGTCGAGGAGTTGCACGCGGCGCCGCGTGATCATCCATTGCGCCGCGCCCCCTTTCAGGCGCCAGTTCAACTACGCCGGATCAGTTTCGATACCTGGTTCGGCATGGGCGTATCGAACGTAATCGCGTTCTTCATCGTGCTGACCGCTGCCGCGACCCTCCATGCGCATCACATCGTCGTCAAGACATCGGCCGACGCGGCGCGTGCACTCGAACCGTTGGCGGGCCACTTCGCCTACGTCCTGTTCGCACTCGGCATCATCGGCACCGGCTTGCTGGCGCTTCCGGTGCTGGCCGGCTCGACCGCCTATGCGATGGCGGGGACCTTCAAATGGAGAAACAGTCTCGCGTTACGCGCGACGGTCGCGGGAGAGTTCTATGCGGTCATCGCTGCCGCGGTCGCGATTGGACTGGTCATGACGTTCGTTCATTTCGACCCGATTCGAGCGCTCTACTGGAGCGCCGTCATCAATGGGATCACGGCAGTGCCCATCATGGTGGTGATGATGCTGATGGCGCAGAGCCGGCGCGTGATGGGCGAGTTCGCAATCAGAGGACCGCTTGCGTGGGGAGGGTGGCTCGCGACGCTCGCCATGGCGCTCGCGGCGGCCGGAATGCTGCTGCCGGGATGAGCCGGCAATCCGGATGGAGAATGCGCATGCCCGCGACGCACCGGCGACGCCTCGCCGGACGGCGGGCGTCGCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCGAGCGCTCCATCGACGACGGTGGCGGCCACGCCCCGGAATTCGACATGCCTGCATCCTCCGATACGGCGAACCGGCGGGCGTCATCAATCGCGCGCATCCAGCGCGGGCTGAAGCGCGGGCTCGGCCGGCGCTGCCGGTTCATGGCCGCCGTGGCGCGCGGCGGT

---
2. What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence? Is there more than one longest or shortest sequence? What are their identifiers? 
---

In [38]:
file_path = "data/dna2.fasta"
fasta = readFasta(file_path)
def minMax(fasta_dict):
    '''Given fasta as input provide Minmum and Maximum Length for Sequence as Dict'''
    seqs = {name: len(seq)for name, seq in fasta_dict.items()}
    mx,mn = max(seqs.values()), min(seqs.values())
    return {key:value for key,value in seqs.items() if value == mx or value == mn}

minMax(fasta)

{'gi|142022655|gb|EQ086233.1|255': 4894, 'gi|142022655|gb|EQ086233.1|346': 115}

---
3. In molecular biology, a reading frame is a way of dividing the DNA sequence of nucleotides into a set of consecutive, non-overlapping triplets (or codons). Depending on where we start, there are six possible reading frames: three in the forward (5' to 3') direction and three in the reverse (3' to 5'). For instance, the three possible forward reading frames for the sequence `AGGTGACACCGCAAGCCTTATATTAGC` are: 

    `AGG TGA CAC CGC AAG CCT TAT ATT AGC`

    `A GGT GAC ACC GCA AGC CTT ATA TTA GC`

    `AG GTG ACA CCG CAA GCC TTA TAT TAG C` 

    These are called reading frames 1, 2, and 3 respectively. An open reading frame (ORF) is the part of a reading frame that has the potential to encode a protein. It starts with a start codon (ATG), and ends with a stop codon (`TAA`, `TAG` or `TGA`). For instance, ATGAAATAG is an ORF of length 9.

    Given an input reading frame on the forward strand (1, 2, or 3) your program should be able to identify all ORFs present in each sequence of the FASTA file, and answer the following questions: what is the length of the longest ORF in the file? What is the identifier of the sequence containing the longest ORF? For a given sequence identifier, what is the longest ORF contained in the sequence represented by that identifier? What is the starting position of the longest ORF in the sequence that contains it? The position should indicate the character number in the sequence. For instance, the following ORF in reading frame 1:
    ```
    >sequence1
    ATGCCCTAG
    ```
    starts at position 1.

    Note that because the following sequence:
    ```
    >sequence2
    ATGAAAAAA
    ```
    does not have any stop codon in reading frame 1, we do not consider it to be an ORF in reading frame 1. 

---


In [72]:
# dict version
def reverseComplemnet(dna):
    transtab = str.maketrans("ATCG", "TAGC")
    return dna.translate(transtab)[::-1].strip()
dna  =  reverseComplemnet('AGGTGACACCGCAAGCCTTATATTAGC')


def orfs(dna,start= 'ATG',stop = ['TAA', 'TAG' , 'TGA'],frame=0) :  
    _orfs = {1:set(),2:set(),3:set()}
    for i in range(0,len(dna)-2,1):
        if dna[i:i+3] == start:
            orf = ''
            j  = i
            while j+3 < len(dna)-1:
                if dna[j:j+3] in stop:
                    orf += dna[j:j+3]
                    _orfs[(i % 3) + 1].add(orf)
                    break
                else:
                    orf += dna[j:j+3]
                j += 3
    return _orfs if frame == 0 else {frame : _orfs[frame]}


def find_orfs(dna, frame = 0, strand= 'b',translate = True):
    forword_orfs = set.union(*orfs(dna,frame = frame).values())
    reverse_orfs = set.union(*orfs(reverseComplemnet(dna),frame = frame).values())
    
    if strand == 'f':
        _orfs = forword_orfs
    elif strand == 'r':
        _orfs = reverse_orfs
    elif strand == 'b':
        _orfs = forword_orfs.union(reverse_orfs)
    else:
        return []
    if translate:
        from Bio.Seq import Seq
        _translated = set()
        for dna in _orfs:
            dna = Seq(dna)
            _translated.add(str(dna.translate()))
        return list(_translated)
    else:
        return list(_orfs)

#dna = 'AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAGTAG'
dna = fasta['gi|142022655|gb|EQ086233.1|45']
dna


'CGTGCTCGGCACGACTATCAGCCCGTATCTGTTTTTCTGGCAGGCCTCCCAGGAAGTCGAGGAGTTGCACGCGGCGCCGCGTGATCATCCATTGCGCCGCGCCCCCTTTCAGGCGCCAGTTCAACTACGCCGGATCAGTTTCGATACCTGGTTCGGCATGGGCGTATCGAACGTAATCGCGTTCTTCATCGTGCTGACCGCTGCCGCGACCCTCCATGCGCATCACATCGTCGTCAAGACATCGGCCGACGCGGCGCGTGCACTCGAACCGTTGGCGGGCCACTTCGCCTACGTCCTGTTCGCACTCGGCATCATCGGCACCGGCTTGCTGGCGCTTCCGGTGCTGGCCGGCTCGACCGCCTATGCGATGGCGGGGACCTTCAAATGGAGAAACAGTCTCGCGTTACGCGCGACGGTCGCGGGAGAGTTCTATGCGGTCATCGCTGCCGCGGTCGCGATTGGACTGGTCATGACGTTCGTTCATTTCGACCCGATTCGAGCGCTCTACTGGAGCGCCGTCATCAATGGGATCACGGCAGTGCCCATCATGGTGGTGATGATGCTGATGGCGCAGAGCCGGCGCGTGATGGGCGAGTTCGCAATCAGAGGACCGCTTGCGTGGGGAGGGTGGCTCGCGACGCTCGCCATGGCGCTCGCGGCGGCCGGAATGCTGCTGCCGGGATGAGCCGGCAATCCGGATGGAGAATGCGCATGCCCGCGACGCACCGGCGACGCCTCGCCGGACGGCGGGCGTCGCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCATTCGCCGAGCGCTCCATCGACGACGGTGGCGGCCACGCCCCGGAATTCGACATGCCTGCATCCTCCGATACGGCGAACCGGCGGGCGTCATCAATCGCGCGCATCCAGCGCGGGCTGAAGCGCGGGCTCGGCCGGCGCTGCCGGTTCATGGCCGCCGTGGCGCGCGGCGGT

In [21]:
# using regex
import re
pattern = re.compile(r'(?=(ATG(?:...)*?)(?=TAG|TGA|TAA))')
pattern.findall(dna)

dna = 'AGCCATGTAGCTAACTCAGGTTACATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAAGCCTGAATGATCCGAGTAGCATCTCAGTAG'
find_orfs(dna,frame=2,translate=True)

['ATGAGGGTGGGG']

In [34]:
import re

def find_all_proteins(dna):
    p = []  
    for seq in [dna, reverseComplemnet(dna)]:
        starts = [aa.start() for aa in re.finditer('ATG', dna)]
        stops =  [aa.start() for aa in re.finditer('TAG|TGA|TAA', dna)]

        for x in starts:
            for y in stops:
                if (y-x) > 0 and ((y-x) % 3) == 0:  
                    p.append(dna[x:y+3])
                    break
    return set(p)

In [36]:
find_all_proteins(dna)

{'ATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA',
 'ATGATCCGAGTAGCATCTCAGTAG',
 'ATGGGGATGACCCCGCGACTTGGATTAGAGTCTCTTTTGGAATAA',
 'ATGTAG'}

In [69]:
def longest_orf(orfs):
    orf_length = [len(orf) for orf in orfs]
    try :
        max_len = max(orf_length)
    except ValueError:
        max_len = 0
    return max_len 

longest_orf(find_orfs(fasta['gi|142022655|gb|EQ086233.1|45'],translate=False,frame=2))        

2268

In [71]:
{seq_name:longest_orf(find_orfs(dna,frame=2,translate= False))  for seq_name,dna in fasta.items()}

{'gi|142022655|gb|EQ086233.1|91': 1632,
 'gi|142022655|gb|EQ086233.1|304': 0,
 'gi|142022655|gb|EQ086233.1|255': 1185,
 'gi|142022655|gb|EQ086233.1|45': 2268,
 'gi|142022655|gb|EQ086233.1|396': 1338,
 'gi|142022655|gb|EQ086233.1|250': 1341,
 'gi|142022655|gb|EQ086233.1|322': 156,
 'gi|142022655|gb|EQ086233.1|88': 171,
 'gi|142022655|gb|EQ086233.1|594': 33,
 'gi|142022655|gb|EQ086233.1|293': 1233,
 'gi|142022655|gb|EQ086233.1|75': 504,
 'gi|142022655|gb|EQ086233.1|454': 822,
 'gi|142022655|gb|EQ086233.1|16': 1458,
 'gi|142022655|gb|EQ086233.1|584': 240,
 'gi|142022655|gb|EQ086233.1|4': 963,
 'gi|142022655|gb|EQ086233.1|277': 336,
 'gi|142022655|gb|EQ086233.1|346': 0,
 'gi|142022655|gb|EQ086233.1|527': 1014}

In [43]:
{seq_name:longest_orf(find_orfs(dna,frame=2,translate=False))  for seq_name,dna in fasta.items()}

{'gi|142022655|gb|EQ086233.1|91': 1632,
 'gi|142022655|gb|EQ086233.1|304': 0,
 'gi|142022655|gb|EQ086233.1|255': 1185,
 'gi|142022655|gb|EQ086233.1|45': 2268,
 'gi|142022655|gb|EQ086233.1|396': 1338,
 'gi|142022655|gb|EQ086233.1|250': 1341,
 'gi|142022655|gb|EQ086233.1|322': 156,
 'gi|142022655|gb|EQ086233.1|88': 171,
 'gi|142022655|gb|EQ086233.1|594': 33,
 'gi|142022655|gb|EQ086233.1|293': 1233,
 'gi|142022655|gb|EQ086233.1|75': 504,
 'gi|142022655|gb|EQ086233.1|454': 822,
 'gi|142022655|gb|EQ086233.1|16': 1458,
 'gi|142022655|gb|EQ086233.1|584': 240,
 'gi|142022655|gb|EQ086233.1|4': 963,
 'gi|142022655|gb|EQ086233.1|277': 336,
 'gi|142022655|gb|EQ086233.1|346': 0,
 'gi|142022655|gb|EQ086233.1|527': 1014}

---
4. A repeat is a substring of a DNA sequence that occurs in multiple copies (more than one) somewhere in the sequence. Although repeats can occur on both the forward and reverse strands of the DNA sequence, we will only consider repeats on the forward strand here. Also we will allow repeats to overlap themselves. For example, the sequence ACACA contains two copies of the sequence ACA - once at position 1 (index 0 in Python), and once at position 3. Given a length n, your program should be able to identify all repeats of length n in all sequences in the FASTA file. Your program should also determine how many times each repeat occurs in the file, and which is the most frequent repeat of a given length.
---